# 邏輯迴歸及其損失函數 (Logistic Loss)

在這個實驗中，你將會：

- 探索為什麼平方誤差損失不適用於邏輯迴歸

- 探索邏輯迴歸的損失函數

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

try:
    from IPython import get_ipython

    ip = get_ipython()
    if ip is not None:
        try:
            ip.run_line_magic('matplotlib', 'widget')
        except Exception:
            ip.run_line_magic('matplotlib', 'inline')
            print("Colab 不支援 matplotlib widget")
except Exception:
    pass

#region 匯入資料
def find_repo_root(marker="README.md"):
    cur = Path.cwd()
    while cur != cur.parent:  # 防止無限迴圈，到達檔案系統根目錄就停
        if (cur / marker).exists():
            return cur
        cur = cur.parent
    return None


def import_data_from_github():
    import os, urllib.request, pathlib, shutil
    
    def isRunningInColab() -> bool:
        return "google.colab" in sys.modules

    def isRunningInJupyterLab() -> bool:
        try:
            import jupyterlab
            return True
        except ImportError:
            return False
        
    def detect_env():
        from IPython import get_ipython
        if isRunningInColab():
            return "Colab"
        elif isRunningInJupyterLab():
            return "JupyterLab"
        elif "notebook" in str(type(get_ipython())).lower():
            return "Jupyter Notebook"
        else:
            return "Unknown"
        
    def get_utils_dir(env): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return "/content/utils"
        else:
            return Path.cwd() / "utils"

    env = detect_env()
    UTILS_DIR = get_utils_dir(env)
    REPO_DIR = "Machine-Learning-Lab"

    #shutil.rmtree(UTILS_DIR, ignore_errors=True)
    os.makedirs(UTILS_DIR, exist_ok=True)

    BASE = f"https://raw.githubusercontent.com/mz038197/{REPO_DIR}/main"
    urllib.request.urlretrieve(f"{BASE}/utils/lab_utils_common_classification.py", f"{UTILS_DIR}/lab_utils_common_classification.py")
    urllib.request.urlretrieve(f"{BASE}/utils/plt_logistic_loss.py", f"{UTILS_DIR}/plt_logistic_loss.py")
    urllib.request.urlretrieve(f"{BASE}/utils/deeplearning.mplstyle", f"{UTILS_DIR}/deeplearning.mplstyle")


repo_root = find_repo_root()

if repo_root is None:
    import_data_from_github()
    repo_root = Path.cwd()
    

os.chdir(repo_root)
print(f"✅ 切換工作目錄至 {Path.cwd()}")
sys.path.append(str(repo_root)) if str(repo_root) not in sys.path else None
print(f"✅ 加入到系統路徑")

from utils.plt_logistic_loss import  plt_logistic_cost, plt_two_logistic_loss_curves, plt_simple_example
from utils.plt_logistic_loss import soup_bowl, plt_logistic_squared_error

plt.style.use('utils/deeplearning.mplstyle')
print("✅ 匯入模組及設定繪圖樣式")
#endregion 匯入資料


<br>

## 邏輯迴歸可以用平方誤差嗎？

回想在 **線性** 迴歸中，我們使用的是 **平方誤差成本函數**：

單一變數時，平方誤差成本的式子為：

  $$J(w,b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2 \tag{1}$$ 
 
其中 

  $$f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{2}$$


回想一下，平方誤差成本有個很好的性質：沿著成本函數的導數（梯度）方向下降，會走到最小值。

In [ ]:
soup_bowl()

這個成本函數在線性迴歸中表現良好，因此也很自然會想把它用在邏輯迴歸上。

不過，如上圖所示，現在的 $f_{w,b}(x)$ 包含一個非線性成分——sigmoid 函數：$f_{w,b}(x^{(i)}) = sigmoid(wx^{(i)} + b )$。

我們試著把平方誤差成本套用到先前實驗的例子上（這次加入 sigmoid）。

以下是訓練資料：

In [ ]:
x_train = np.array([0., 1, 2, 3, 4, 5],dtype=np.longdouble)
y_train = np.array([0,  0, 0, 1, 1, 1],dtype=np.longdouble)
plt_simple_example(x_train, y_train)

現在，讓我們用 **平方誤差成本** 來畫出成本的曲面圖：

  $$J(w,b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2 $$ 
 
其中 

  $$f_{w,b}(x^{(i)}) = sigmoid(wx^{(i)} + b )$$


In [ ]:
plt.close('all')
plt_logistic_squared_error(x_train,y_train)
plt.show()

雖然這張圖很有趣，但上面的曲面遠不如線性迴歸的「湯碗（soup bowl）」那樣平滑！

邏輯迴歸需要更符合其非線性特性的成本函數。這會從 **損失函數（loss function）** 開始，下面會說明。

<br>

## 邏輯損失函數

邏輯迴歸（Logistic Regression）使用更適合**分類**任務的損失函數，因為目標值是 0 或 1，而不是任意實數。

>**名詞定義：** 在本課程中，採用以下定義：  
**Loss（損失）**：衡量「單一樣本」的預測與目標值之差。  
**Cost（成本）**：衡量「整個訓練集」的損失彙總結果。

定義如下：

- $loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)})$ 表示單一資料點的損失（loss）：

\begin{equation}
  loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) = \begin{cases}
    - \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) & \text{if $y^{(i)}=1$}\\
    - \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) & \text{if $y^{(i)}=0$}
  \end{cases}
\end{equation}

- $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ 是模型的預測值，而 $y^{(i)}$ 是目標值（標記）。

- $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = g(\mathbf{w} \cdot\mathbf{x}^{(i)}+b)$，其中 $g$ 是 sigmoid 函數。

- 記號慣例：`log` 表示自然對數（natural logarithm）。

這個損失函數的關鍵特色在於它使用了兩條不同的曲線：當目標為 0（$y=0$）時使用一條，當目標為 1（$y=1$）時使用另一條。

把兩者合併後，就能呈現我們希望的行為：當預測與目標一致時損失為 0；當預測偏離目標時，損失會快速增大。請看下圖：

In [ ]:
plt_two_logistic_loss_curves()

把兩條曲線合併後，外觀會有點像平方誤差損失的二次曲線。注意，x 軸是 $f_{\mathbf{w},b}$，也就是 sigmoid 的輸出；而 sigmoid 的輸出嚴格介於 0 與 1 之間。

上面的損失函數可以改寫成較容易實作的形式：

$$loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) = -y^{(i)} \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right)$$

這個式子乍看之下很嚇人，但只要記得 $y^{(i)}$ 只能取兩個值：0 或 1，就會容易許多。我們可以把它分成兩種情況來看：

當 $y^{(i)} = 0$ 時，左邊那一項會被消去：

$$
\begin{align}
loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), 0) &= (-(0) \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - 0\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) \\
&= -\log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right)
\end{align}
$$

而當 $y^{(i)} = 1$ 時，右邊那一項會被消去：

$$
\begin{align}
  loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), 1) &=  (-(1) \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - 1\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right)\\
  &=  -\log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right)
\end{align}
$$

有了這個新的邏輯損失函數後，我們就能把所有樣本的損失彙整起來，形成成本函數（cost function）——這會是下一個實驗的主題。

現在，先來看看我們上面那個簡單例子中，「成本」對「參數」的曲線：

In [ ]:
plt.close('all')
cst = plt_logistic_cost(x_train,y_train)

這條曲線很適合用梯度下降來最佳化！它沒有平台區、局部極小值或不連續點。注意，它不像平方誤差那樣是個「碗狀」。

圖中同時畫出成本以及成本的對數，是為了讓你看清楚：當成本很小時，曲線仍然有斜率，並且會繼續下降。

提醒：你可以用滑鼠旋轉上面的圖。

<br>

## 恭喜！

你已經：

- 確認平方誤差損失函數不適合用在分類任務

- 推導並觀察了 **適合** 分類任務的邏輯損失函數。

